In [2]:
# melissa robertson February 2022
# UCSF - no price data
# NY Presbyterian - done
# Cedars Sinai - no DRG data
#nyu-langone



# Dependencies
import pandas as pd
import numpy as np
import json
import os
import csv
import re


In [ ]:
################################################################################################################

In [ ]:
def categorise(row):
    if  'BCBS'.lower() in row['Plan'].lower():
        return 'BCBS'
    elif  'Blue'.lower() in row['Plan'].lower():
        return 'BCBS'
    elif 'Cigna'.lower() in row['Plan'].lower():
        return 'Cigna'
    elif 'Aetna'.lower() in row['Plan'].lower():
        return 'Aetna'
    elif 'Cash'.lower() in row['Plan'].lower():
        return 'Self Pay'
    elif 'Gross Charge'.lower() in row['Plan'].lower():
        return'Gross Charge'
    elif 'United'.lower() in row['Plan'].lower():
        return 'United Healthcare'
    else:
        return 'Other'

In [121]:
#read in UCSF data
filepath = os.path.join("../943281657_ucsf-medical-center_standardcharges.json")
with open(filepath) as jsonfile:
    UCSF_json = pd.read_json(jsonfile)

In [122]:
#subset to DRG only
UCSF_drg=UCSF_json[UCSF_json['Code Type'] =='DRG']

In [123]:
#remove unneccissary columsn
UCSF_drg=UCSF_drg.drop(['Revenue Code','CPT/HCPCS','Default Modifier','CPT/HCPCS Numeric','IP De-Identified Minimum Negotiated Charge','IP De-Identified Maximum Negotiated Charge','OP De-Identified Minimum Negotiated Charge',
       'OP De-Identified Maximum Negotiated Charge'], axis=1)

In [124]:
UCSF_drg.columns

Index(['Code', 'Code Type', 'Description', 'Default Gross Charge',
       'Emergency Care Gross Charge', 'Default Gross Clinic Charge',
       'Hospital Ancillary Global Component Gross Charge',
       'Hospital Based Professional Services Gross Charge',
       'US/MANAGED CHOICE AETNA  IP Negotiated Charge',
       'US/ELECT CHOICE AETNA  IP Negotiated Charge',
       ...
       'TRICARE WEST ACTIVE HNET FED SVCS  OP Negotiated Charge',
       'APPLE INC UHC CHOICE PLUS  OP Negotiated Charge',
       'ADMIN UNITED HLTHCR  OP Negotiated Charge',
       'CHOICE PLUS UNITED HLTHCR  OP Negotiated Charge',
       'ADMIN UNITED HLTHCR (UMR)  OP Negotiated Charge',
       'UNITED HLTHCR SELECT/ PLUS  OP Negotiated Charge',
       'UNITED HLTHCR SELECT/ PLUS POS  OP Negotiated Charge',
       'HILL PHYS MED GRP/SAC WHA  OP Negotiated Charge',
       'SF MIA OP Negotiated Charge',
       'ANTHEM BLUE CONNECTION EPO ANTHEM OP Negotiated Charge'],
      dtype='object', length=210)

In [125]:
#rename columns
UCSF_drg=UCSF_drg.rename(columns={"Code": "DRG Number", "Description": "DRG Description",'Code Type':'Billing Code Type'}) 

In [127]:
#check to see if any charges
UCSF_drg=UCSF_drg.melt(id_vars=['DRG Number', 'Billing Code Type', 'DRG Description'], 
        var_name="Plan", 
        value_name="Charge Amount")

In [110]:
UCSF_drg['Plan'].unique()


array(['Default Gross Charge', 'Emergency Care Gross Charge',
       'Default Gross Clinic Charge',
       'Hospital Ancillary Global Component Gross Charge',
       'Hospital Based Professional Services Gross Charge',
       'IP De-Identified Minimum Negotiated Charge',
       'IP De-Identified Maximum Negotiated Charge',
       'OP De-Identified Minimum Negotiated Charge',
       'OP De-Identified Maximum Negotiated Charge',
       'US/MANAGED CHOICE AETNA  IP Negotiated Charge',
       'US/ELECT CHOICE AETNA  IP Negotiated Charge',
       'US HLTH/OPEN CHOICE AETNA  IP Negotiated Charge',
       'BX PRUDENT BUYER ANTHEM  IP Negotiated Charge',
       'BX CA ANTHEM  IP Negotiated Charge',
       'BX/BS OUT OF STATE BLUE CARD PROGRAM  IP Negotiated Charge',
       'FEDERAL EMPLOYEES/BS ANTHEM  IP Negotiated Charge',
       'PERSCHOICE/PERSCARE ANTHEM  IP Negotiated Charge',
       'GOOGLE HEALTHCARE ANTHEM  IP Negotiated Charge',
       'DELTA HLTH SYSTEMS ANTHEM  IP Negotiated Charge

In [128]:
# create payer column
UCSF_drg['Payor'] = UCSF_drg.apply(lambda row: categorise(row), axis=1)

In [129]:
UCSF_drg['Payor'].unique()

array(['Gross Charge', 'Other', 'Aetna', 'BCBS', 'Cigna',
       'United Healthcare'], dtype=object)

In [130]:
#add additonal columns
UCSF_drg["US News Ranking"]=9
UCSF_drg["Hospital Name"]="UCSF Medical Center"

In [134]:
# reorder columns
UCSF_drg = UCSF_drg[["US News Ranking","Hospital Name", "Billing Code Type","DRG Number", "DRG Description","Payor","Plan","Charge Amount"]]

In [135]:
# ouput csv file
UCSF_drg.to_csv("UCSF Medical Center.csv", index=False)

In [ ]:
#NO charges available for UCSF

In [ ]:
######################################################################################################################

In [ ]:
## NY presbyterian file
#########################################################################################################################

In [10]:
#NYpresbyterian
NYpres_json=pd.read_json("../133957095_NewYorkPresbyterianHospital_updated01132022.json",lines=True,orient='records')


In [11]:
NYpres_json.shape

(2, 5757)

In [12]:
NYpres_json_long=NYpres_json.T
NYpres_json_long[0]

0       {'Code (CPT/DRG)': 1110000001, 'Description': ...
1       {'Code (CPT/DRG)': 1110000004, 'Description': ...
2       {'Code (CPT/DRG)': 1110000005, 'Description': ...
3       {'Code (CPT/DRG)': 1110000006, 'Description': ...
4       {'Code (CPT/DRG)': 1110000007, 'Description': ...
                              ...                        
5752    {'Code (CPT/DRG)': 982, 'Description': 'EXTENS...
5753    {'Code (CPT/DRG)': 983, 'Description': 'EXTENS...
5754    {'Code (CPT/DRG)': 987, 'Description': 'NON-EX...
5755    {'Code (CPT/DRG)': 988, 'Description': 'NON-EX...
5756    {'Code (CPT/DRG)': 989, 'Description': 'NON-EX...
Name: 0, Length: 5757, dtype: object

In [13]:
NYpresdict=[]
i=0
j=0
while j <2:
    for each in NYpres_json_long[j]:
        NYpresdict.append(each)
       # print(each)
        i+=1
    j+=1
    
    

In [14]:
#keep only DRG
NYpresdrg_dict=[]
for each in NYpresdict:
    try:
        if each['Code (CPT/DRG)']<1000:
            NYpresdrg_dict.append(each)
    except:
        continue
     
        

In [16]:
len(NYpresdrg_dict)


756

In [75]:
NYpresdf = pd.DataFrame.from_dict(NYpresdrg_dict)

In [76]:
NYpresdf.columns

Index(['Code (CPT/DRG)', 'Description', 'Rev Code', 'Inpatient/Outpatient',
       'Gross Charges', 'Discounted Cash Price', 'Aetna', 'Cigna', 'Empire',
       'Emblem', 'United', 'Aetna Medicare', 'AgeWell Medicare',
       'Empire Medicare', 'Fidelis Medicare', 'Emblem Medicare',
       'Healthfirst Medicare', 'UHC Community Plan/United Medicare',
       'VNS Medicare', 'WellCare Medicare', '1199',
       'Affinity Molina Medicaid/CHP', 'Affinity Molina Essential',
       'Amida Care Medicaid', 'Empire Healthplus Medicaid',
       'Empire Healthplus Essential', 'Empire Healthplus Exchange',
       'Fidelis Medicaid/CHP', 'Fidelis Essential/Exchange',
       'Healthfirst Essential/Exchange', 'Healthfirst Medicaid/CHP',
       'Emblem Medicaid/CHP', 'MVP Medicaid/CHP', 'MVP Essential',
       'United Commuity Plan Medicaid', 'United Commuity Plan Essential',
       'VNS Medicaid', 'Minimum Negotiated Charge',
       'Maximum Negotiated Charge'],
      dtype='object')

In [77]:
#remove unneccissary columsn
NYpresdf=NYpresdf.drop(["Rev Code","Inpatient/Outpatient",'Minimum Negotiated Charge','Maximum Negotiated Charge'], axis=1)

In [78]:
NYpresdf.columns

Index(['Code (CPT/DRG)', 'Description', 'Gross Charges',
       'Discounted Cash Price', 'Aetna', 'Cigna', 'Empire', 'Emblem', 'United',
       'Aetna Medicare', 'AgeWell Medicare', 'Empire Medicare',
       'Fidelis Medicare', 'Emblem Medicare', 'Healthfirst Medicare',
       'UHC Community Plan/United Medicare', 'VNS Medicare',
       'WellCare Medicare', '1199', 'Affinity Molina Medicaid/CHP',
       'Affinity Molina Essential', 'Amida Care Medicaid',
       'Empire Healthplus Medicaid', 'Empire Healthplus Essential',
       'Empire Healthplus Exchange', 'Fidelis Medicaid/CHP',
       'Fidelis Essential/Exchange', 'Healthfirst Essential/Exchange',
       'Healthfirst Medicaid/CHP', 'Emblem Medicaid/CHP', 'MVP Medicaid/CHP',
       'MVP Essential', 'United Commuity Plan Medicaid',
       'United Commuity Plan Essential', 'VNS Medicaid'],
      dtype='object')

In [79]:
#rename columns
NYpresdf=NYpresdf.rename(columns={"Code (CPT/DRG)": "DRG Number", "Description": "DRG Description",}) 

In [80]:
# convert columns to rows
NYpresdf=NYpresdf.melt(id_vars=["DRG Number", "DRG Description"], 
        var_name="Plan", 
        value_name="Charge Amount")

In [81]:
#add additonal columns
NYpresdf["US News Ranking"]=7
NYpresdf["Hospital Name"]="New York-Presbyterian Hospital"
NYpresdf["Billing Code Type"]="MSDRG"

In [82]:
NYpresdf[NYpresdf['Plan'].str.contains('Cigna')]

,DRG Number,DRG Description,Plan,Charge Amount,US News Ranking,Hospital Name,Billing Code Type
2268,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Cigna,1024812.054,7,New York-Presbyterian Hospital,MSDRG
2269,2,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Cigna,1024812.054,7,New York-Presbyterian Hospital,MSDRG
2270,3,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Cigna,746260.830,7,New York-Presbyterian Hospital,MSDRG
2271,4,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Cigna,465692.850,7,New York-Presbyterian Hospital,MSDRG
2272,5,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,Cigna,544148.325,7,New York-Presbyterian Hospital,MSDRG
...,...,...,...,...,...,...,...
3019,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Cigna,99079.596,7,New York-Presbyterian Hospital,MSDRG
3020,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Cigna,64538.838,7,New York-Presbyterian Hospital,MSDRG
3021,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Cigna,127956.654,7,New York-Presbyterian Hospital,MSDRG
3022,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Cigna,66651.984,7,New York-Presbyterian Hospital,MSDRG


In [115]:
# create payer column
NYpresdf['Payor'] = NYpresdf.apply(lambda row: categorise(row), axis=1)

In [116]:
#check that it worked
NYpresdf['Payor'].unique()

array(['Gross Charge', 'Self Pay', 'Aetna', 'Cigna', 'Other',
       'United Healthcare'], dtype=object)

In [86]:
NYpresdf = NYpresdf[["US News Ranking","Hospital Name", "Billing Code Type","DRG Number", "DRG Description","Payor","Plan","Charge Amount"]]

In [88]:
# ouput csv file
NYpresdf.to_csv('New York-Presbyterian.csv', index=False)

In [ ]:
#########################################################################################################################

In [ ]:
#read in Cedars Sinai data;

In [ ]:
##########################################################################################################################

In [44]:
file_path = "../951644600_cedars-sinai-medical-center_standardcharges.csv"
cedars_sinai_df=pd.read_csv(file_path)

In [45]:
cedars_sinai_df.columns

Index(['Inpatient/Outpatient', 'CHARGE_CODE', 'CHARGE_DESC', 'MEDICATION_NAME',
       'ERX_ID', 'NDC_CODE', 'CPT_CODE', 'GROSS_CHARGES', 'MIN', 'MAX',
       'Aetna_HMO', 'Aetna_PPO', 'Blue_Cross_HMO_PPO', 'Blue_Cross_SR_HMO',
       'Blue_Shield_HMO_PPO', 'Blue_Shield_Sr_HMO', 'Cigna_PPO',
       'Healthnet_HMO', 'Healthnet_PPO', 'Kaiser', 'Kaiser_SR_MMC',
       'Multiplan', 'UHC_HMO', 'UHC_PPO', 'UHC_SR_HMO/PPO',
       'Uninsured rate for patients that do not qualify for financial assistance*'],
      dtype='object')

In [47]:
cedars_sinai_df['Inpatient/Outpatient'].unique()

array(['CSMC IP', 'CSMC OPED',
       "*Cedars-Sinai does not have a standard cash price for inpatient services because each patient's specific medical and financial situation is different. If you are uninsured, the price you will pay for inpatient care at Cedars-Sinai will depend on the services you need and the amount of financial assistance you qualify for under our general financial assistance policy.   Although there are no standard cash prices for inpatient services, if you are uninsured and would like information on rates for services, please contact Patient Financial Services at 866-803-1777. We're committed to providing quality healthcare to everyone in the community who needs it. This includes offering full financial assistance or services at discounted prices to eligible people who cannot afford to pay for part or all of their care. Payment plans are also available."],
      dtype=object)

In [49]:
inpatient_df=cedars_sinai_df[cedars_sinai_df['Inpatient/Outpatient']=='CSMC IP']

In [55]:
inpatient_df['CHARGE_CODE'].unique()

array([ 1200001.,  1200002.,  1200003., ...,  9900515.,  9900517.,
       80016217.])

In [1]:
inpatient_df['CHARGE_CODE']<1000:

SyntaxError: invalid syntax (Temp/ipykernel_9656/2189078099.py, line 1)

In [415]:
inpatient_df.dtypes

Inpatient/Outpatient                                                          object
CHARGE_CODE                                                                  float64
CHARGE_DESC                                                                   object
MEDICATION_NAME                                                               object
ERX_ID                                                                       float64
NDC_CODE                                                                      object
CPT_CODE                                                                      object
GROSS_CHARGES                                                                float64
MIN                                                                          float64
MAX                                                                          float64
Aetna_HMO                                                                    float64
Aetna_PPO                                                        

In [418]:
inpatient_df['CHARGE_DESC'].str.contains('DRG')

<bound method Series.unique of 0       False
1       False
2       False
3       False
4       False
        ...  
9094    False
9095    False
9096    False
9097    False
9098    False
Name: CHARGE_DESC, Length: 9099, dtype: bool>

In [284]:
files=np.arange(13)
filestart="../133971298_nyu-langone-health_standardcharges_"
filenames=[]

In [285]:
###########################################
#read in all 12 files from NYU langone and keep only DRG rows
for item in files:
    name=f"NYU_{item}"
    try: 
         exec(f'NYU_{item}=pd.read_csv(f"{filestart}{item}.csv", low_memory=False)')
         exec(f"NYU_drg_{item}=NYU_{item}[NYU_{item}[' Billing Code'].str.contains('MSDRG', na=False)]") 
         #exec(f"print(NYU_drg_{item}.shape)")
         exec(f"filenames.append(f'NYU_drg_{item}')")
    except: 
        continue

In [ ]:
filenames

In [388]:
starter_cols=NYU_drg_1[NYU_drg_1.columns[1:3]]

In [389]:
starter_cols.columns

Index([' Billing Code', 'Identifier Description'], dtype='object')

In [390]:
for file in filenames:
    print(file)

NYU_drg_1
NYU_drg_2
NYU_drg_3
NYU_drg_4
NYU_drg_5
NYU_drg_6
NYU_drg_7
NYU_drg_8
NYU_drg_9
NYU_drg_10
NYU_drg_11
NYU_drg_12


In [391]:
#keep columns with Blue Cross Blues Shield, United Health Care, Cigna, Health Partners, Aetna
newfilenames=[]
providers=['BCBS', 'UNITED', 'CIGNA', 'AETNA' ]
for file in filenames:
    column_names=[]
    for column in eval(file):
        for provider in providers:
            if provider in column:
               # print(column)
                column_names.append(column)
    try:
        exec(f"new_{file} = {file}[column_names]")
        exec(f"newfilenames.append(f'new_{file}')")
    except:
        print(f"error {file}")


In [392]:
for file in newfilenames:
    print(file)

new_NYU_drg_1
new_NYU_drg_2
new_NYU_drg_3
new_NYU_drg_4
new_NYU_drg_5
new_NYU_drg_6
new_NYU_drg_7
new_NYU_drg_8
new_NYU_drg_9
new_NYU_drg_10
new_NYU_drg_11
new_NYU_drg_12


In [393]:
frames=[starter_cols,new_NYU_drg_1,
new_NYU_drg_2,
new_NYU_drg_3,
new_NYU_drg_4,
new_NYU_drg_5,
new_NYU_drg_6,
new_NYU_drg_7,
new_NYU_drg_8,
new_NYU_drg_9,
new_NYU_drg_10,
new_NYU_drg_11,
new_NYU_drg_12]
result = pd.concat(frames,axis=1)

In [394]:
result.shape

(767, 427)

In [395]:
result

,Billing Code,Identifier Description,Hospital A1005AETNA HMO,Hospital B1005AETNA HMO,Hospital C1005AETNA HMO,Hospital D1005AETNA HMO,Hospital A1006AETNA INDEMNITY,Hospital B1006AETNA INDEMNITY,Hospital C1006AETNA INDEMNITY,Hospital D1006AETNA INDEMNITY,...,Hospital C1773 UNITED HEALTHCARE CHOICE NSLIJ,Hospital D1773 UNITED HEALTHCARE CHOICE NSLIJ,Hospital A3911 UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY),Hospital B3911 UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY),Hospital C3911 UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY),Hospital D3911 UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY),Hospital A1352 UNITED HEALTHCARE OPTUM HEALTH,Hospital B1352 UNITED HEALTHCARE OPTUM HEALTH,Hospital C1352 UNITED HEALTHCARE OPTUM HEALTH,Hospital D1352 UNITED HEALTHCARE OPTUM HEALTH
46236,MSDRG001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,618308.78,618308.78,475737.79,576211.16,1981340.18,1981340.18,1981340.18,1981340.18,...,630278.85,569127.43,601875,601875,630278.85,569127.43,601875,601875,630278.85,569127.43
46237,MSDRG002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,320135.59,320135.59,246318.03,298339.12,1321224.39,1321224.39,1321224.39,1321224.39,...,326333.21,294671.45,601875,601875,326333.21,294671.45,601875,601875,326333.21,294671.45
46238,MSDRG003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,408571.12,408571.12,314361.9,380753.51,2053176.55,2053176.55,2053176.55,2053176.55,...,416480.79,376072.66,513766,513766,416480.79,376072.66,513766,513766,416480.79,376072.66
46239,MSDRG004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,254962.66,254962.66,196172.82,237603.5,1413751.59,1413751.59,1413751.59,1413751.59,...,259898.58,234682.49,320607.95,320607.95,259898.58,234682.49,320607.95,320607.95,259898.58,234682.49
46240,MSDRG005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,218875.48,218875.48,168406.69,203973.32,1619771.81,1619771.81,1619771.81,1619771.81,...,223112.77,201465.74,275229.39,275229.39,223112.77,201465.74,275229.39,275229.39,223112.77,201465.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46998,MSDRG987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,70055.12,70055.12,53901.66,65285.41,342572.2,342572.2,342572.2,342572.2,...,71411.34,64482.82,88092.23,88092.23,71411.34,64482.82,88092.23,88092.23,71411.34,64482.82
46999,MSDRG988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,36491.36,36491.36,28077.11,34006.85,130678.26,130678.26,130678.26,130678.26,...,37197.81,33588.78,45886.8,45886.8,37197.81,33588.78,45886.8,45886.8,37197.81,33588.78
47000,MSDRG989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,24028.19,24028.19,18487.71,22392.22,82114.68,82114.68,82114.68,82114.68,...,24493.36,22116.94,30214.73,30214.73,24493.36,22116.94,30214.73,30214.73,24493.36,22116.94
47001,MSDRG998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
